<a href="https://colab.research.google.com/github/Hettieboo/AuctionDimensionProcessor/blob/main/Final_Piasa_Extraction_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

# Upload your Excel file
uploaded = files.upload()

Saving calculateur convelio 2302 16.10.25 (3).xlsx to calculateur convelio 2302 16.10.25 (3).xlsx


In [ ]:
# PIASA AUCTION DIMENSION PROCESSOR
# ==============================================

!pip install openpyxl

import pandas as pd
import re
import numpy as np
from typing import List, Dict, Optional, Tuple
from google.colab import files

class AuctionDimensionProcessor:
    def __init__(self):
        self.multiples_keywords = {
            "paire": 2, "deux": 2, "trois": 3, "quatre": 4,
            "cinq": 5, "six": 6, "sept": 7, "huit": 8,
            "neuf": 9, "dix": 10, "onze": 11, "douze": 12,
            "treize": 13, "quatorze": 14, "quinze": 15,
            "seize": 16, "dix-sept": 17, "dix-huit": 18,
            "dix-neuf": 19, "vingt": 20
        }

        # Track reclassifications for reporting
        self.reclassified_lots = []

        self.dimension_patterns = {
            'H': re.compile(r'H\s*[:\s]*(\d+(?:[.,]\d+)?)', re.IGNORECASE),
            'L': re.compile(r'L\s*[:\s]*(\d+(?:[.,]\d+)?)', re.IGNORECASE),
            'P': re.compile(r'P\s*[:\s]*(\d+(?:[.,]\d+)?)', re.IGNORECASE),
            'D': re.compile(r'D\s*[:\s]*(\d+(?:[.,]\d+)?)\s*(?:cm|×|x)', re.IGNORECASE),
            'Diameter': re.compile(r'Ø\s*(?:\([^)]*\))?\s*[:\s]*(\d+(?:[.,]\d+)?)', re.IGNORECASE),
        }

        self.complete_pattern = re.compile(
            r'H\s*[:\s]*(\d+(?:[.,]\d+)?)\s*[×x]\s*(?:L\s*[:\s]*(\d+(?:[.,]\d+)?)\s*[×x]\s*P\s*[:\s]*(\d+(?:[.,]\d+)?)|Ø\s*[:\s]*(\d+(?:[.,]\d+)?))',
            re.IGNORECASE
        )

        self.simple_3d_pattern = re.compile(r'(\d+(?:[.,]\d+)?)\s*[×x]\s*(\d+(?:[.,]\d+)?)\s*[×x]\s*(\d+(?:[.,]\d+)?)\s*cm', re.IGNORECASE)

        self.two_d_pattern = re.compile(r'(\d+(?:[.,]\d+)?)\s*[×x]\s*(\d+(?:[.,]\d+)?)\s*cm', re.IGNORECASE)

        self.true_2d_materials = [
            'huile', 'gouache', 'aquarelle', 'acrylique', 'pastel', 'crayon',
            'dessin', 'gravure', 'lithographie', 'sérigraphie', 'estampe',
            'papier', 'toile', 'canvas', 'encre', 'fusain', 'sanguine', 'collage',
            'painting', 'drawing', 'print', 'watercolor', 'tapis', 'carpet', 'rug',
            'tirage', 'photographie', 'photographique', 'photograph', 'cibachrome',
            'argentique', 'gélatine', 'gelatin', 'c-print', 'chromogenic', 'chromogenique',
            'épreuve', 'numérique', 'pigmentaire', 'inkjet', 'giclée', 'digital',
            'fumage'
        ]

        # Keywords indicating physical objects on panels/canvas (should be 3D with D=5)
        self.panel_object_keywords = ['panneau', 'panel']

        # Keywords indicating assemblages/3D mixed media with objects (should be 3D with D=5)
        self.assemblage_keywords = ['objets', 'objects', 'assemblage', 'relief', 'montage', 'boite', 'boîte', 'box', 'caisse']

        self.force_3d_keywords = ['valise', 'suitcase', 'malle', 'table', 'chaise', 'meuble', 'furniture']

        self.fashion_keywords = ['robe', 'trench', 'veste', 'pantalon', 'costume', 'jupe', 'manteau',
                                'chaussures', 'sac', 'taille', 'size']

        self.complex_keywords = ['tiroir', 'drawer', 'miroir', 'mirror', 'siège', 'compartiment',
                                'exceptionnel', 'numéroté', 'édition', 'limited']

        # Material keywords for extraction
        self.material_keywords = {
            'cuivre': 'Copper',
            'laiton': 'Brass',
            'bronze': 'Bronze',
            'fer': 'Iron',
            'acier': 'Steel',
            'aluminium': 'Aluminum',
            'métal': 'Metal',
            'metal': 'Metal',
            'bois': 'Wood',
            'chêne': 'Oak',
            'noyer': 'Walnut',
            'teck': 'Teak',
            'palissandre': 'Rosewood',
            'ébène': 'Ebony',
            'châtaignier': 'Chestnut',
            'verre': 'Glass',
            'cristal': 'Crystal',
            'céramique': 'Ceramic',
            'porcelaine': 'Porcelain',
            'grès': 'Stoneware',
            'faïence': 'Earthenware',
            'marbre': 'Marble',
            'pierre': 'Stone',
            'granit': 'Granite',
            'cuir': 'Leather',
            'textile': 'Textile',
            'tissu': 'Fabric',
            'velours': 'Velvet',
            'soie': 'Silk',
            'coton': 'Cotton',
            'lin': 'Linen',
            'plastique': 'Plastic',
            'résine': 'Resin',
            'plexiglas': 'Plexiglass',
            'toile': 'Canvas',
            'papier': 'Paper',
            'carton': 'Cardboard',
            'wood': 'Wood',
            'oak': 'Oak',
            'walnut': 'Walnut',
            'teak': 'Teak',
            'glass': 'Glass',
            'crystal': 'Crystal',
            'ceramic': 'Ceramic',
            'porcelain': 'Porcelain',
            'marble': 'Marble',
            'stone': 'Stone',
            'leather': 'Leather',
            'fabric': 'Fabric',
            'velvet': 'Velvet',
            'silk': 'Silk',
            'cotton': 'Cotton'
        }

    def normalize_number(self, num_str: str) -> Optional[float]:
        if not num_str:
            return None
        try:
            return float(num_str.replace(',', '.'))
        except:
            return None

    def extract_material(self, text: str) -> str:
        """Extract material from TYPESET text and return in English"""
        if not isinstance(text, str):
            return ''

        text_lower = text.lower()
        found_materials = []

        for french_material, english_material in self.material_keywords.items():
            if french_material in text_lower:
                found_materials.append(english_material)

        # Remove duplicates while preserving order
        seen = set()
        unique_materials = []
        for material in found_materials:
            if material not in seen:
                seen.add(material)
                unique_materials.append(material)

        return ', '.join(unique_materials) if unique_materials else ''

    def should_skip_lot(self, text: str) -> bool:
        """Skip lots marked Ouvert or Fermé"""
        if not isinstance(text, str):
            return False
        text_lower = text.lower()
        return bool(re.search(r'\b(ouvert|fermé|ferme)\b', text_lower))

    def detect_item_count(self, text: str) -> Tuple[int, List[str]]:
        """Detect item count with word boundaries and context awareness"""
        if not isinstance(text, str):
            return 1, []

        text_lower = text.lower()
        flags = []

        # Check for "chaque" as standalone word - flag for manual review
        if re.search(r'\bchaque\b', text_lower):
            flags.append("CHAQUE_DETECTED: manual count verification needed")
            # Still try to detect base count for reference
            for word, count in self.multiples_keywords.items():
                if re.search(r'\b' + word + r'\b', text_lower):
                    return count, flags
            return 1, flags

        # Ensemble de X pattern (most reliable)
        match = re.search(r'ensemble\s+de\s+(\d+)', text_lower)
        if match:
            try:
                return int(match.group(1)), flags
            except:
                pass

        # Look for quantity indicators at START of description
        # "Paire de", "Suite de trois", etc.
        for word, count in self.multiples_keywords.items():
            # Must be at start or after punctuation/newline, followed by "de"
            pattern = r'(?:^|[.\n])\s*' + word + r'\s+de\s+'
            if re.search(pattern, text_lower):
                # Check if it's NOT an edition reference
                edition_pattern = r'(?:édition|edition|tirage)\s+de\s+' + word
                if not re.search(edition_pattern, text_lower):
                    return count, flags

        # Fallback: look for standalone multiples NOT followed by object descriptors
        # Avoid "seize bras", "quatre pieds", etc.
        # Also avoid edition references like "édition de trois"
        object_words = r'(?:bras|pied|pieds|jambe|jambes|branches|arms|legs|feet)'
        edition_words = r'(?:édition|edition|tirage|exemplaires|exemplaire|copies|copy)'

        for word, count in self.multiples_keywords.items():
            pattern = r'\b' + word + r'\b'
            if re.search(pattern, text_lower):
                # Check if followed by object descriptor or edition reference
                context_pattern = r'\b' + word + r'\s+' + object_words
                edition_context = r'(?:édition|edition|tirage)\s+de\s+' + word + r'\s+(?:' + edition_words + r')'

                if not re.search(context_pattern, text_lower) and not re.search(edition_context, text_lower):
                    return count, flags

        return 1, flags

    def extract_dimensions(self, text: str) -> List[Dict]:
        """Extract all dimension sets from text"""
        if not isinstance(text, str):
            return []

        text = text.replace('\n', ' ').replace('\xa0', ' ').strip()
        dimensions = []
        segments = re.split(r'[;\n]', text)

        for segment in segments:
            segment = segment.strip()
            if not segment:
                continue

            # Try complete patterns with H/L/P labels first
            matches = self.complete_pattern.findall(segment)
            if matches:
                for match in matches:
                    h, l, p, diameter = match
                    dimensions.append({
                        'H': self.normalize_number(h),
                        'L': self.normalize_number(l) if l else None,
                        'P': self.normalize_number(p) if p else None,
                        'Diameter': self.normalize_number(diameter) if diameter else None
                    })
                continue

            # Try simple 3D pattern (number × number × number cm) - MUST come before 2D
            matches_3d = self.simple_3d_pattern.finditer(segment)
            found_3d = False
            for match_3d in matches_3d:
                dimensions.append({
                    'H': self.normalize_number(match_3d.group(1)),
                    'L': self.normalize_number(match_3d.group(2)),
                    'P': self.normalize_number(match_3d.group(3)),
                    'Diameter': None
                })
                found_3d = True

            if found_3d:
                continue

            # Try 2D pattern - THIS IS KEY FOR YOUR CASE
            matches_2d = self.two_d_pattern.finditer(segment)
            for match_2d in matches_2d:
                dimensions.append({
                    'H': self.normalize_number(match_2d.group(1)),
                    'L': self.normalize_number(match_2d.group(2)),
                    'P': None,
                    'Diameter': None
                })

            # Only do individual extraction if no 2D patterns found
            if not self.two_d_pattern.search(segment):
                # Individual dimension extraction
                h_vals = [self.normalize_number(m) for m in self.dimension_patterns['H'].findall(segment)]
                l_vals = [self.normalize_number(m) for m in self.dimension_patterns['L'].findall(segment)]
                p_vals = [self.normalize_number(m) for m in self.dimension_patterns['P'].findall(segment)]
                d_vals = [self.normalize_number(m) for m in self.dimension_patterns['Diameter'].findall(segment)]

                max_len = max(len(h_vals), len(l_vals), len(p_vals), len(d_vals))
                if max_len > 0:
                    for i in range(max_len):
                        dimensions.append({
                            'H': h_vals[i] if i < len(h_vals) else None,
                            'L': l_vals[i] if i < len(l_vals) else None,
                            'P': p_vals[i] if i < len(p_vals) else None,
                            'Diameter': d_vals[i] if i < len(d_vals) else None
                        })

        # Filter out completely empty
        return [d for d in dimensions if any(v is not None for v in d.values())]

    def classify_item_type(self, text: str, dims: List[Dict]) -> Tuple[str, List[str]]:
        """Classify as 2D/3D/MANUAL_CHECK with minimal flagging"""
        flags = []
        text_lower = text.lower()[:400]

        # Fashion items with Taille - manual check only
        if 'taille' in text_lower:
            flags.append("FASHION_ITEM")
            return 'MANUAL_CHECK', flags

        # Curtains with "paire" - ambiguous count (pair vs individual panels)
        if 'rideau' in text_lower and 'paire' in text_lower:
            flags.append("CURTAIN_PAIR_COUNT")
            return 'MANUAL_CHECK', flags

        # Books with dimension extractions - likely false positives from volume numbers
        if 'livre' in text_lower and len(dims) > 0:
            flags.append("BOOK_DIMENSION_CHECK")
            return 'MANUAL_CHECK', flags

        # Items using "D" notation for depth - flag for manual review
        if re.search(r'd\s+\d+\s*cm', text_lower):
            flags.append("D_NOTATION_DEPTH")
            return 'MANUAL_CHECK', flags

        # Rugs with L×P pattern (no H) - manual review for shipping method
        is_rug = any(kw in text_lower for kw in ['tapis', 'carpet', 'rug'])
        has_l_p_only = any(d['L'] and d['P'] and not d['H'] and not d['Diameter'] for d in dims)

        if is_rug and has_l_p_only:
            flags.append("RUG_L_P_PATTERN")
            return 'MANUAL_CHECK', flags

        # Check dimension complexity
        has_3d = any(d['H'] and d['L'] and d['P'] for d in dims)
        has_diameter = any(d['Diameter'] for d in dims)
        has_2d_only = any(d['H'] and d['L'] and not d['P'] and not d['Diameter'] for d in dims)
        has_height_only = any(d['H'] and not d['L'] and not d['P'] and not d['Diameter'] for d in dims)

        # Flag items with only height dimension - incomplete dimensions for shipping
        if has_height_only:
            flags.append("HEIGHT_ONLY_OBJECT")
            return 'MANUAL_CHECK', flags

        # Count unique dimension format types
        formats = []
        if has_3d:
            formats.append('HLP')
        if has_diameter:
            formats.append('HD')
        if has_2d_only:
            formats.append('HL')

        # Complex lots - only flag if MULTIPLE materials + compartments/drawers
        complex_indicators = ['tiroir', 'drawer', 'compartiment', 'miroir', 'mirror']
        material_indicators = ['cuir', 'métal', 'bronze', 'toile', 'leather', 'metal']

        complex_count = sum(1 for kw in complex_indicators if kw in text_lower)
        material_count = sum(1 for kw in material_indicators if kw in text_lower)

        if complex_count >= 2 and material_count >= 3:
            flags.append("COMPLEX_MULTI_PART")
            return 'MANUAL_CHECK', flags

        # Mixed formats = manual check ONLY if also high item count
        if len(formats) > 1:
            item_count, _ = self.detect_item_count(text_lower)
            if item_count > 5:
                flags.append(f"MIXED_FORMATS_HIGH_COUNT")
                return 'MANUAL_CHECK', flags
            # Otherwise just proceed with mixed formats (take first found)

        # Full 3D dimensions (H×L×P) = always 3D
        if has_3d:
            return '3D', flags

        # H×Ø = 3D (standard cylindrical object)
        if has_diameter:
            return '3D', flags

        # Check for objects on panels/canvas - should be 3D even with only H×L
        if has_2d_only:
            # Check for assemblage keywords first
            has_assemblage = any(kw in text_lower for kw in self.assemblage_keywords)

            if has_assemblage:
                flags.append("ASSEMBLAGE_3D_MANUAL_CHECK")
                return '3D', flags

            # Check for panel keywords
            has_panel = any(kw in text_lower for kw in self.panel_object_keywords)

            # Panel is 3D ONLY if it's NOT a traditional painting technique
            # Traditional painting = huile/aquarelle/acrylique/gouache/pastel sur panneau
            traditional_painting_on_panel = has_panel and any(
                material in text_lower for material in ['huile sur', 'aquarelle sur', 'acrylique sur',
                                                         'gouache sur', 'pastel sur', 'oil on', 'watercolor on']
            )

            # If it's a traditional painting on panel, treat as 2D
            if traditional_painting_on_panel:
                return '2D', flags

            # NEW REFINEMENT: "technique mixte sur toile/canvas" without assemblage keywords = 2D painting
            # Only treat as 3D if it has assemblage keywords OR is on panel without toile
            has_canvas_or_toile = 'toile' in text_lower or 'canvas' in text_lower
            has_technique_mixte = 'technique mixte' in text_lower or 'mixed media' in text_lower

            if has_technique_mixte and has_canvas_or_toile and not has_assemblage:
                # This is a mixed media painting on canvas, not a 3D relief
                # Let it fall through to 2D material check
                flags.append("TECHNIQUE_MIXTE_TOILE_RECLASSIFIED")
                pass  # Continue to 2D material check below
            elif has_panel and has_technique_mixte:
                # Technique mixte on panel (without toile) = likely 3D sculptural
                flags.append("PANEL_OBJECT_3D")
                return '3D', flags

            # Otherwise check if it's a true 2D artwork
            for material in self.true_2d_materials:
                if material in text_lower:
                    return '2D', flags

            # Check for "A DECADRER" or "DECADRER" - indicates framed 2D artwork
            if 'decadrer' in text_lower or 'à décadrer' in text_lower:
                flags.append("DECADRER_2D")
                return '2D', flags

            # HL format but no 2D material = probably 3D missing P
            return '3D', flags

        return '3D', flags

    def process_lot(self, row: pd.Series) -> Dict:
        """Process a single LOT - returns one row with H_1, L_1, D_1... columns"""
        typeset = str(row.get('TYPESET', ''))
        lot_number = row.get('LOT', 'Unknown')

        # Initialize result early
        result = row.to_dict()

        # Check for Ouvert/Fermé - flag for manual review instead of skipping
        if self.should_skip_lot(typeset):
            result['ITEM_COUNT'] = 1
            result['ITEM_TYPE'] = 'MANUAL_CHECK'
            result['MANUAL_REVIEW_REQUIRED'] = True
            result['MATERIAL'] = self.extract_material(typeset)
            result['PROCESSING_FLAGS'] = 'OUVERT/FERMÉ: excluded from dimension processing'
            result['CONVERSION_LOG'] = 'LOT marked Ouvert or Fermé - requires manual review'
            return result

        # Extract dimensions
        dims = self.extract_dimensions(typeset)

        # Detect item count
        item_count, count_flags = self.detect_item_count(typeset)

        # Initialize result - move this earlier
        multiple_dims_flag = False

        # --- CRITICAL: Single item with multiple dimension sets -> manual check ---
        if item_count == 1 and len(dims) > 1:
            all_flags = count_flags.copy()
            all_flags.append("MULTIPLE_DIMENSIONS_SINGLE_ITEM")
            multiple_dims_flag = True

            # Find the largest dimension set (by volume/area)
            largest_dim = max(dims, key=lambda d: (
                (d.get('H') or 0) * (d.get('L') or 0) * (d.get('P') or 1) * (d.get('Diameter') or 1)
            ))

            # Keep only the largest dimension set
            dims = [largest_dim]

            result['ITEM_TYPE'] = 'MANUAL_CHECK'
            result['MANUAL_REVIEW_REQUIRED'] = True
            result['ITEM_COUNT'] = item_count
            result['MATERIAL'] = self.extract_material(typeset)
            result['PROCESSING_FLAGS'] = '; '.join(all_flags)
            result['CONVERSION_LOG'] = "Single item with multiple dimension sets - using largest dimensions only"

        # Classify type
        item_type, type_flags = self.classify_item_type(typeset, dims)

        # Merge flags - preserve MULTIPLE_DIMENSIONS flag if it exists
        if not multiple_dims_flag:
            all_flags = count_flags + type_flags
        else:
            all_flags.extend(type_flags)

        conversion_log = []

        # Determine final item type
        if multiple_dims_flag:
            item_type_to_use = 'MANUAL_CHECK'
        else:
            item_type_to_use = item_type

        # Check if assemblage flag exists - if so, require manual review
        has_assemblage_flag = any("ASSEMBLAGE_3D_MANUAL_CHECK" in flag for flag in all_flags)
        if has_assemblage_flag:
            item_type_to_use = 'MANUAL_CHECK'

        # Rule: >10 items = manual check, no expansion
        if item_count > 10:
            all_flags.append(f"HIGH_COUNT: {item_count}>10")
            item_type_to_use = 'MANUAL_CHECK'

        # Update result with final values (only if not already set by multiple dims logic)
        if not multiple_dims_flag:
            result['ITEM_TYPE'] = item_type_to_use
            result['ITEM_COUNT'] = item_count
            result['MANUAL_REVIEW_REQUIRED'] = (item_type_to_use == 'MANUAL_CHECK')
            result['MATERIAL'] = self.extract_material(typeset)

        # Books with fake dimensions - skip all dimension processing
        if any("BOOK_DIMENSION_CHECK" in flag for flag in all_flags):
            result['PROCESSING_FLAGS'] = '; '.join(all_flags)
            result['CONVERSION_LOG'] = "Book lot - volume numbers extracted as fake dimensions, skipped processing"
            return result

        # Check for flags that should trigger manual review even if ITEM_TYPE is not MANUAL_CHECK
        review_trigger_flags = [
            "CHAQUE_DETECTED",
            "TECHNIQUE_MIXTE_TOILE_RECLASSIFIED",
            "DECADRER_2D",
            "NO_DIMENSIONS"
        ]

        for trigger_flag in review_trigger_flags:
            if any(trigger_flag in flag for flag in all_flags):
                result['MANUAL_REVIEW_REQUIRED'] = True
                break

        # Track if this lot was reclassified from technique mixte refinement
        # ONLY track if it actually ended up as 2D (not if it became MANUAL_CHECK for other reasons)
        if any("TECHNIQUE_MIXTE_TOILE_RECLASSIFIED" in flag for flag in all_flags) and result['ITEM_TYPE'] == '2D':
            self.reclassified_lots.append({
                'LOT': lot_number,
                'DESCRIPTION': typeset[:100] + '...' if len(typeset) > 100 else typeset,
                'OLD_TYPE': '3D (panel/technique mixte rule)',
                'NEW_TYPE': '2D (technique mixte on canvas)',
                'REASON': 'Technique mixte sur toile without assemblage keywords'
            })

        # Manual check cases - but still process largest dimension if available
        if result['ITEM_TYPE'] == 'MANUAL_CHECK':
            # Merge flags and logs if they already exist - handle NaN values
            existing_flags = result.get('PROCESSING_FLAGS', '')
            existing_log = result.get('CONVERSION_LOG', '')

            # Convert NaN/float to empty string
            if not isinstance(existing_flags, str) or pd.isna(existing_flags):
                existing_flags = ''
            if not isinstance(existing_log, str) or pd.isna(existing_log):
                existing_log = ''

            new_flags_str = '; '.join(all_flags) if all_flags else ''

            if existing_flags and new_flags_str:
                result['PROCESSING_FLAGS'] = existing_flags + '; ' + new_flags_str
            elif new_flags_str:
                result['PROCESSING_FLAGS'] = new_flags_str
            elif existing_flags:
                result['PROCESSING_FLAGS'] = existing_flags
            else:
                result['PROCESSING_FLAGS'] = ''

            if existing_log and not existing_log.startswith("Single item"):
                result['CONVERSION_LOG'] = existing_log

            # If we have dimensions, still process them
            if not dims or all(all(v is None for v in d.values()) for d in dims):
                return result
            # Otherwise continue to process the largest dimension set

        # No dimensions found but item count > 1
        if not dims and item_count > 1:
            conversion_log.append(f"NO_DIMENSIONS: {item_count} items without measurements")
            result['PROCESSING_FLAGS'] = '; '.join(all_flags)
            result['CONVERSION_LOG'] = '; '.join(conversion_log)
            result['MANUAL_REVIEW_REQUIRED'] = True
            return result

        # No dimensions at all
        if not dims or all(all(v is None for v in d.values()) for d in dims):
            if item_count > 1:
                conversion_log.append(f"NO_DIMENSIONS: {item_count} items without measurements")
            else:
                conversion_log.append("NO_DIMENSIONS: no measurements found")
            result['PROCESSING_FLAGS'] = result.get('PROCESSING_FLAGS', '') or ('; '.join(all_flags) if all_flags else '')
            result['CONVERSION_LOG'] = '; '.join(conversion_log)
            result['MANUAL_REVIEW_REQUIRED'] = True
            return result

        # If fewer dimensions than item count, replicate dimensions
        if dims and len(dims) < item_count:
            # Replicate dimensions to match item count
            while len(dims) < item_count:
                dims.extend(dims[:item_count - len(dims)])
            dims = dims[:item_count]  # Trim to exact count
            conversion_log.append(f"Replicated dimensions to match {item_count} items")

        # Process dimensions into H_1, L_1, D_1, etc.
        for i, dim in enumerate(dims[:item_count], 1):  # Limit to item_count
            h, l, p, diameter = dim['H'], dim['L'], dim['P'], dim['Diameter']

            if item_type == '2D':
                # 2D logic: L=max(H,L), D=5
                if h and l:
                    result[f'H_{i}'] = min(h, l)
                    result[f'L_{i}'] = max(h, l)
                    conversion_log.append(f"Item {i}: L=max(H,L)")
                elif h:
                    result[f'L_{i}'] = h
                elif l:
                    result[f'L_{i}'] = l

                result[f'D_{i}'] = 5.0
                result[f'P_{i}'] = p
                result[f'Diameter_{i}'] = diameter
                conversion_log.append(f"Item {i}: D=5 (2D)")

            else:  # 3D logic
                result[f'H_{i}'] = h
                result[f'P_{i}'] = p
                result[f'Diameter_{i}'] = diameter

                # Special case: Panel objects or assemblages with only H×L (no P given) - assign D=5
                is_panel_object = any("PANEL_OBJECT_3D" in flag for flag in all_flags)
                is_assemblage = any("ASSEMBLAGE_3D_MANUAL_CHECK" in flag for flag in all_flags)

                if h and l and not p and not diameter and (is_panel_object or is_assemblage):
                    result[f'L_{i}'] = l
                    result[f'D_{i}'] = 5.0
                    if is_assemblage:
                        conversion_log.append(f"Item {i}: D=5 (assemblage)")
                    else:
                        conversion_log.append(f"Item {i}: D=5 (panel object)")
                # For cylindrical items (H×Ø), set L from diameter
                elif diameter and not l:
                    result[f'L_{i}'] = diameter
                    result[f'D_{i}'] = diameter
                    conversion_log.append(f"Item {i}: L=Ø, D=Ø")
                elif diameter:
                    result[f'L_{i}'] = l
                    result[f'D_{i}'] = diameter
                    conversion_log.append(f"Item {i}: D=Ø")
                elif p:
                    result[f'L_{i}'] = l if l else p  # Use P as fallback for L when missing
                    result[f'D_{i}'] = p
                    if not l:
                        conversion_log.append(f"Item {i}: L=P (missing L), D=P")
                    else:
                        conversion_log.append(f"Item {i}: D=P")
                elif l:
                    result[f'L_{i}'] = l
                    result[f'D_{i}'] = l
                    conversion_log.append(f"Item {i}: D=L")
                else:
                    result[f'L_{i}'] = l

        # Handle PROCESSING_FLAGS - only set if not already set
        if 'PROCESSING_FLAGS' not in result or not result.get('PROCESSING_FLAGS'):
            result['PROCESSING_FLAGS'] = '; '.join(all_flags) if all_flags else ''

        # Merge conversion logs - handle NaN values and avoid duplication
        existing_log = result.get('CONVERSION_LOG', '')
        new_log = '; '.join(conversion_log) if conversion_log else ''

        # Convert NaN/float to empty string
        if not isinstance(existing_log, str) or pd.isna(existing_log):
            existing_log = ''

        # Only append new_log if it's different from existing_log (avoid duplication)
        if existing_log and new_log and existing_log != new_log:
            result['CONVERSION_LOG'] = existing_log + '; ' + new_log
        elif new_log and not existing_log:
            result['CONVERSION_LOG'] = new_log
        elif existing_log and not new_log:
            result['CONVERSION_LOG'] = existing_log
        else:
            result['CONVERSION_LOG'] = ''

        return result

    def process_dataframe(self, df: pd.DataFrame) -> pd.DataFrame:
        """Process entire dataframe - one row per LOT"""
        print(f"Processing {len(df)} lots...")

        results = []

        for idx, row in df.iterrows():
            if idx % 100 == 0:
                print(f"Progress: {idx}/{len(df)}")

            result = self.process_lot(row)
            results.append(result)

        df_result = pd.DataFrame(results)

        # Add missing columns
        for col in ['ITEM_COUNT', 'ITEM_TYPE', 'PROCESSING_FLAGS', 'CONVERSION_LOG', 'MANUAL_REVIEW_REQUIRED']:
            if col not in df_result.columns:
                if col == 'ITEM_COUNT':
                    df_result[col] = 1
                elif col == 'ITEM_TYPE':
                    df_result[col] = '3D'
                elif col == 'MANUAL_REVIEW_REQUIRED':
                    df_result[col] = False
                else:
                    df_result[col] = ''

        # Reorder columns logically
        base_cols = [c for c in df_result.columns if not c.startswith(('H_', 'L_', 'D_', 'P_', 'Diameter_'))]
        dim_cols = sorted([c for c in df_result.columns if c.startswith(('H_', 'L_', 'D_', 'P_', 'Diameter_'))],
                         key=lambda x: (int(re.search(r'\d+', x).group()), x[0]))

        df_result = df_result[base_cols + dim_cols]

        print(f"\nCompleted! {len(df_result)} lots processed")
        print(f"Manual review required: {df_result['MANUAL_REVIEW_REQUIRED'].sum()}")

        return df_result

# MAIN EXECUTION
print("Starting One-Row-Per-LOT Processing...")

df = pd.read_excel('calculateur convelio 2302 16.10.25 (3).xlsx')
print(f"Loaded {len(df)} rows")

processor = AuctionDimensionProcessor()
df_final = processor.process_dataframe(df)

# Analysis
print("\nProcessing Summary:")
if 'ITEM_TYPE' in df_final.columns:
    print("\nItem Type Distribution:")
    for item_type, count in df_final['ITEM_TYPE'].value_counts().items():
        print(f"  {item_type}: {count}")

if 'ITEM_COUNT' in df_final.columns:
    max_item_count = df_final['ITEM_COUNT'].max()
    print(f"\nMaximum items in a single lot: {max_item_count}")

    # Find which lot has the maximum
    max_lot = df_final[df_final['ITEM_COUNT'] == max_item_count]['LOT'].iloc[0] if len(df_final[df_final['ITEM_COUNT'] == max_item_count]) > 0 else None
    if max_lot:
        print(f"Lot with most items: LOT {max_lot} ({max_item_count} items)")

if 'MANUAL_REVIEW_REQUIRED' in df_final.columns:
    manual_count = df_final['MANUAL_REVIEW_REQUIRED'].sum()
    if manual_count > 0:
        print(f"\nManual Review Reasons (top 5):")
        manual_df = df_final[df_final['MANUAL_REVIEW_REQUIRED'] == True]
        for flag, count in manual_df['PROCESSING_FLAGS'].value_counts().head(5).items():
            print(f"  {flag}: {count}")

# Save and download
filename = 'extracted_dimensions_one_row_per_lot.xlsx'
df_final.to_excel(filename, index=False)
files.download(filename)

print(f"\nDone! File saved as '{filename}'")

Starting One-Row-Per-LOT Processing...
Loaded 86 rows
Processing 86 lots...
Progress: 0/86

Completed! 86 lots processed
Manual review required: 9

Processing Summary:

Item Type Distribution:
  2D: 61
  3D: 19
  MANUAL_CHECK: 6

Maximum items in a single lot: 6
Lot with most items: LOT 16 (6 items)

Manual Review Reasons (top 5):
  MULTIPLE_DIMENSIONS_SINGLE_ITEM; MULTIPLE_DIMENSIONS_SINGLE_ITEM: 3
  MULTIPLE_DIMENSIONS_SINGLE_ITEM; MULTIPLE_DIMENSIONS_SINGLE_ITEM; TECHNIQUE_MIXTE_TOILE_RECLASSIFIED: 1
  : 1
  TECHNIQUE_MIXTE_TOILE_RECLASSIFIED: 1
  CHAQUE_DETECTED: manual count verification needed; MULTIPLE_DIMENSIONS_SINGLE_ITEM; CHAQUE_DETECTED: manual count verification needed; MULTIPLE_DIMENSIONS_SINGLE_ITEM: 1


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Done! File saved as 'extracted_dimensions_one_row_per_lot.xlsx'
